In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


Using TensorFlow backend.


In [ ]:
DATA = "ARTINT"

In [2]:
d_feature_vector = pickle.load(open(DATA+"_train_only_bib_feature_vector.p", "rb"))
d_feature_vector_test = pickle.load(open(DATA+"_train_only_bib_feature_vector.p", "rb"))


In [3]:

x_train = []
y_train = []
for i in d_feature_vector.keys():
    x_train.append(d_feature_vector[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape)


(3056, 8913)


In [4]:

x_test = []
y_test = []
for i in d_feature_vector_test.keys():
    x_test.append(d_feature_vector_test[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_test.append(label)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape)


(3056, 8913)


In [5]:
c = 0
for i in y_test:
    if i == 0.0:
        c = c+ 1
print(c)

1444


In [6]:
# Input - Layer
model = models.Sequential()
model.add(layers.Dense(50, activation = "relu", input_dim=8913))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                445700    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 450,851
Trainable params: 450,851
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile( optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [8]:
results = model.fit( x_train,y_train, epochs= 5, batch_size = 32, validation_data = (x_test, y_test))


Train on 3056 samples, validate on 3056 samples
Epoch 1/5
3056/3056 [==============================] - 2s 688us/step - loss: 0.4382 - acc: 0.7932 - val_loss: 0.2369 - val_acc: 0.8995
Epoch 2/5
3056/3056 [==============================] - 2s 569us/step - loss: 0.2312 - acc: 0.8976 - val_loss: 0.1556 - val_acc: 0.9257
Epoch 3/5
3056/3056 [==============================] - 2s 612us/step - loss: 0.1630 - acc: 0.9257 - val_loss: 0.1216 - val_acc: 0.9385
Epoch 4/5
3056/3056 [==============================] - 2s 580us/step - loss: 0.1307 - acc: 0.9368 - val_loss: 0.1133 - val_acc: 0.9401
Epoch 5/5
3056/3056 [==============================] - 2s 581us/step - loss: 0.1197 - acc: 0.9365 - val_loss: 0.1097 - val_acc: 0.9411


In [9]:
predicted =  model.predict(x_test)

In [10]:
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [11]:
print(f1_score(y_test, predicted, average="macro"))
print(precision_score(y_test, predicted, average="macro"))
print(recall_score(y_test, predicted, average="macro"))    

0.9410988457481051
0.9444725333498427
0.9441326478007739


In [12]:
x = classification_report(y_test,predicted)

In [13]:
print(x)

              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      1444
         1.0       1.00      0.89      0.94      1612

   micro avg       0.94      0.94      0.94      3056
   macro avg       0.94      0.94      0.94      3056
weighted avg       0.95      0.94      0.94      3056



In [14]:
model.save(DATA+'only_bib_model')